# Day 7: Classification Metrics - Confusion Matrix, ROC/PR, Calibration, and Business Costs

**MGMT 47400 - Predictive Analytics**  
**4-Week Online Course**  
**Day 7 - Wednesday May 26, 2027**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/07_classification_metrics_thresholding.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Compute and interpret precision, recall, F1, ROC-AUC, PR-AUC
2. Select thresholds based on business cost tradeoffs
3. Understand calibration and when it matters
4. Handle class imbalance at the evaluation level (metrics first)
5. Produce a metrics dashboard table for model comparison

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score,
    ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
)
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

## 1. Load Data and Train Model

In [ ]:
# Load dataset
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

# Split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED, stratify=y_temp)

# Train model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
])

pipeline.fit(X_train, y_train)
y_pred_val = pipeline.predict(X_val)
y_proba_val = pipeline.predict_proba(X_val)[:, 1]

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
print(f"Validation Accuracy: {pipeline.score(X_val, y_val):.4f}")

## 2. The Confusion Matrix Deep Dive

### Understanding the 2×2 Matrix

```
                    Predicted Negative    Predicted Positive
Actual Negative          TN                     FP
Actual Positive          FN                     TP
```

**Definitions:**
- **True Positive (TP)**: Correctly predicted positive
- **True Negative (TN)**: Correctly predicted negative
- **False Positive (FP)**: Incorrectly predicted positive (Type I error)
- **False Negative (FN)**: Incorrectly predicted negative (Type II error)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_val, y_pred_val)

fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Malignant (0)', 'Benign (1)'])
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Validation Set')
plt.tight_layout()
plt.show()

TN, FP, FN, TP = cm.ravel()
print("=== CONFUSION MATRIX VALUES ===")
print(f"True Negatives (TN):  {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"True Positives (TP):  {TP}")

## 3. Core Classification Metrics

### Precision
**Precision = TP / (TP + FP)**
- "Of all positive predictions, how many were correct?"
- High precision = few false alarms

### Recall (Sensitivity, True Positive Rate)
**Recall = TP / (TP + FN)**
- "Of all actual positives, how many did we find?"
- High recall = few missed positives

### F1 Score
**F1 = 2 × (Precision × Recall) / (Precision + Recall)**
- Harmonic mean of precision and recall
- Useful when you need balance

### Accuracy
**Accuracy = (TP + TN) / (TP + TN + FP + FN)**
- Overall correctness
- ⚠️ Misleading with class imbalance!

In [ ]:
# Calculate all metrics
metrics = {
    'Accuracy': (TP + TN) / (TP + TN + FP + FN),
    'Precision': TP / (TP + FP) if (TP + FP) > 0 else 0,
    'Recall': TP / (TP + FN) if (TP + FN) > 0 else 0,
    'F1': f1_score(y_val, y_pred_val),
    'Specificity': TN / (TN + FP) if (TN + FP) > 0 else 0
}

print("=== CLASSIFICATION METRICS ===")
for metric, value in metrics.items():
    print(f"{metric:15s}: {value:.4f}")

# Using sklearn functions
print("\n=== SKLEARN CLASSIFICATION REPORT ===")
print(classification_report(y_val, y_pred_val, target_names=['Malignant', 'Benign']))

## 4. ROC Curve and AUC

### ROC (Receiver Operating Characteristic)
- Plots True Positive Rate vs False Positive Rate
- Shows performance across all thresholds
- AUC (Area Under Curve) summarizes performance

**AUC Interpretation:**
- AUC = 1.0: Perfect classifier
- AUC = 0.5: Random guessing
- AUC < 0.5: Worse than random (predictions inverted)

In [ ]:
# ROC Curve
fpr, tpr, thresholds_roc = roc_curve(y_val, y_proba_val)
roc_auc = roc_auc_score(y_val, y_proba_val)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"ROC-AUC Score: {roc_auc:.4f}")
print("\n💡 Higher AUC = better separation between classes")

## 5. Precision-Recall Curve

### When to Use PR Curve vs ROC

**Use ROC when:**
- Classes are balanced
- You care about both TPR and FPR

**Use PR Curve when:**
- Classes are imbalanced
- Positive class is more important
- You want to focus on precision/recall tradeoff

In [ ]:
# Precision-Recall Curve
precision, recall, thresholds_pr = precision_recall_curve(y_val, y_proba_val)
pr_auc = average_precision_score(y_val, y_proba_val)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, linewidth=2, label=f'PR Curve (AP = {pr_auc:.3f})')
baseline = y_val.sum() / len(y_val)
plt.axhline(y=baseline, color='k', linestyle='--', linewidth=1, label=f'Baseline (class ratio = {baseline:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average Precision (PR-AUC): {pr_auc:.4f}")
print("\n💡 Closer to top-right = better model")

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Build a threshold sweep and pick a threshold by business cost.

**Scenario:** Medical diagnosis
- False Negative (missed cancer): Cost = $50,000 (late treatment)
- False Positive (false alarm): Cost = $1,000 (unnecessary biopsy)

---

In [ ]:
# Threshold sweep with cost analysis
cost_FN = 50000  # Cost of missing a malignant case
cost_FP = 1000   # Cost of false alarm

thresholds_to_test = np.linspace(0.1, 0.9, 50)
threshold_results = []

for thresh in thresholds_to_test:
    y_pred_thresh = (y_proba_val >= thresh).astype(int)
    cm = confusion_matrix(y_val, y_pred_thresh)
    TN, FP, FN, TP = cm.ravel()
    
    # Calculate metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    # Calculate expected cost
    total_cost = (FN * cost_FN) + (FP * cost_FP)
    
    threshold_results.append({
        'Threshold': thresh,
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN,
        'Precision': precision,
        'Recall': recall,
        'Total_Cost': total_cost
    })

threshold_df = pd.DataFrame(threshold_results)

# Find optimal threshold (minimum cost)
optimal_idx = threshold_df['Total_Cost'].idxmin()
optimal_thresh = threshold_df.loc[optimal_idx, 'Threshold']
optimal_cost = threshold_df.loc[optimal_idx, 'Total_Cost']

print("=== OPTIMAL THRESHOLD BY COST ===")
print(f"Optimal Threshold: {optimal_thresh:.3f}")
print(f"Expected Cost: ${optimal_cost:,.0f}")
print(f"\nAt this threshold:")
print(f"  Precision: {threshold_df.loc[optimal_idx, 'Precision']:.4f}")
print(f"  Recall: {threshold_df.loc[optimal_idx, 'Recall']:.4f}")
print(f"  FP: {threshold_df.loc[optimal_idx, 'FP']:.0f}")
print(f"  FN: {threshold_df.loc[optimal_idx, 'FN']:.0f}")

# Visualize cost vs threshold
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(threshold_df['Threshold'], threshold_df['Total_Cost'], linewidth=2)
axes[0].axvline(x=optimal_thresh, color='r', linestyle='--', label=f'Optimal = {optimal_thresh:.3f}')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Expected Cost ($)')
axes[0].set_title('Total Cost vs Threshold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(threshold_df['Threshold'], threshold_df['Precision'], label='Precision', linewidth=2)
axes[1].plot(threshold_df['Threshold'], threshold_df['Recall'], label='Recall', linewidth=2)
axes[1].axvline(x=optimal_thresh, color='r', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('Score')
axes[1].set_title('Precision-Recall vs Threshold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### YOUR ANALYSIS:

**Question 1: Why is the optimal threshold different from 0.5?**  
[Your answer - think about asymmetric costs]

**Question 2: What happens if you change the cost ratio?**  
[Your answer - try FN cost = $100,000]

**Question 3: In production, how would you monitor this?**  
[Your answer - what could go wrong over time?]

---

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Explain why accuracy fails under imbalance (with evidence).

---

In [ ]:
# Create imbalanced scenario
# Simulate: 95% benign, 5% malignant
from sklearn.datasets import make_classification

X_imb, y_imb = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    weights=[0.95, 0.05],  # 95% class 0, 5% class 1
    random_state=RANDOM_SEED
)

X_train_imb, X_val_imb, y_train_imb, y_val_imb = train_test_split(
    X_imb, y_imb, test_size=0.3, random_state=RANDOM_SEED, stratify=y_imb
)

# Train model
pipe_imb = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
])
pipe_imb.fit(X_train_imb, y_train_imb)

y_pred_imb = pipe_imb.predict(X_val_imb)

# Compare metrics
print("=== IMBALANCED DATASET (95% class 0, 5% class 1) ===")
print(f"\nClass distribution:")
print(pd.Series(y_val_imb).value_counts(normalize=True))

print(f"\n=== MODEL PERFORMANCE ===")
print(classification_report(y_val_imb, y_pred_imb, target_names=['Class 0 (95%)', 'Class 1 (5%)']))

# Naive baseline: always predict majority class
y_naive = np.zeros_like(y_val_imb)
naive_acc = (y_naive == y_val_imb).mean()

print(f"\n=== NAIVE BASELINE (always predict 0) ===")
print(f"Accuracy: {naive_acc:.4f}")
print(f"Recall for class 1: 0.0000 (missed all positives!)")

print(f"\n💡 Key Insight: {naive_acc*100:.1f}% accuracy by predicting everything as class 0!")
print(f"💡 This is why accuracy alone is dangerous with imbalance")

### YOUR EXPLANATION:

**Why accuracy is misleading:**  
[Your explanation with evidence from above]

**Better metrics for imbalance:**  
[Which metrics would you use instead?]

**Real-world example:**  
[Give an example where this matters]

---

## 6. Metrics Dashboard

In [ ]:
# Complete metrics dashboard
def create_metrics_dashboard(y_true, y_pred, y_proba):
    """Generate comprehensive classification metrics"""
    cm = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = cm.ravel()
    
    metrics = {
        'Accuracy': (TP + TN) / (TP + TN + FP + FN),
        'Precision': TP / (TP + FP) if (TP + FP) > 0 else 0,
        'Recall': TP / (TP + FN) if (TP + FN) > 0 else 0,
        'F1': f1_score(y_true, y_pred),
        'Specificity': TN / (TN + FP) if (TN + FP) > 0 else 0,
        'ROC_AUC': roc_auc_score(y_true, y_proba),
        'PR_AUC': average_precision_score(y_true, y_proba),
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN
    }
    return metrics

dashboard = create_metrics_dashboard(y_val, y_pred_val, y_proba_val)

print("=== COMPREHENSIVE METRICS DASHBOARD ===")
print(f"\nPerformance Metrics:")
print(f"  Accuracy:    {dashboard['Accuracy']:.4f}")
print(f"  Precision:   {dashboard['Precision']:.4f}")
print(f"  Recall:      {dashboard['Recall']:.4f}")
print(f"  F1 Score:    {dashboard['F1']:.4f}")
print(f"  Specificity: {dashboard['Specificity']:.4f}")
print(f"  ROC-AUC:     {dashboard['ROC_AUC']:.4f}")
print(f"  PR-AUC:      {dashboard['PR_AUC']:.4f}")

print(f"\nConfusion Matrix:")
print(f"  TP: {dashboard['TP']:4d}    FP: {dashboard['FP']:4d}")
print(f"  FN: {dashboard['FN']:4d}    TN: {dashboard['TN']:4d}")

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Confusion Matrix**: Foundation for understanding classification errors
2. **Precision vs Recall**: Tradeoff between false positives and false negatives
3. **ROC and PR Curves**: Visualize performance across thresholds
4. **Cost-Based Thresholding**: Align decisions to business objectives
5. **Imbalance Handling**: Accuracy is dangerous - use precision, recall, AUC

### Critical Rules:

> **"Never trust accuracy alone"**

> **"Choose thresholds based on business costs, not defaults"**

> **"With imbalance, use PR curves over ROC curves"**

### Next Steps:

- Day 8: Cross-validation for robust model comparison
- We'll use today's metrics with proper CV evaluation
- Apply to your project dataset

---

## Bibliography

- Fawcett, T. (2006). "An introduction to ROC analysis." *Pattern Recognition Letters*, 27(8), 861-874.
- Saito, T., & Rehmsmeier, M. (2015). "The precision-recall plot is more informative than the ROC plot when evaluating binary classifiers on imbalanced datasets." *PLOS ONE*.
- scikit-learn User Guide: [Classification metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)
- Provost, F., & Fawcett, T. (2013). *Data Science for Business* - Chapter on evaluation and costs

---

**End of Day 7 Notebook**